# MACHINE LEARNING MODEL -  IROHACK PROJECT 

## STEP 1: Update the Dataset

#### STEP 1.1: Load Libraries

In [13]:
import pandas as pd
import numpy as np
import polars as pl


### 📌 STEP 1: Update the Dataset

In [ ]:
import yfinance as yf
import polars as pl
import pandas as pd
import time
from datetime import datetime, timedelta

# 📂 File paths
final_dataset_path = r"D:\Project\TradeVision\Data\Final Dataset\Processing.csv"

# 📥 Load the main dataset
TradeVision = pl.read_csv(final_dataset_path, try_parse_dates=True)

# 📆 Define the update period (e.g., last 10 days)
start_date = (datetime.today() - timedelta(days=10)).strftime('%Y-%m-%d')
end_date = datetime.today().strftime('%Y-%m-%d')

# 📌 Get all unique tickers from the dataset (S&P 500)
tickers = TradeVision["Ticker"].unique().to_list()

# 📌 Add Ibovespa tickers
ibov_tickers = [
    "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "ABEV3.SA",
    "BBAS3.SA", "B3SA3.SA", "BRFS3.SA", "CSNA3.SA", "ELET3.SA"
]  # Add more tickers as needed

# 🔄 Merge both lists (avoiding duplicates)
tickers = list(set(tickers + ibov_tickers))

# 📊 Create a list to store new data
new_data = []

# 🔄 Download stock data in batches to avoid rate limits
try:
    print(f"Fetching data for {len(tickers)} tickers...")

    # Download data for all tickers at once
    data = yf.download(tickers, start=start_date, end=end_date, progress=False, group_by="ticker")

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            company_name = stock.info.get("longName", "Unknown")  # Get company name

            # Extract stock data
            df = data[ticker].reset_index()
            df["Ticker"] = ticker
            df["Company"] = company_name

            # Select relevant columns
            df = df[["Date", "Ticker", "Company", "Open", "High", "Low", "Close", "Volume"]]
            df.columns = ["Date", "Ticker", "Company", "Open_Price", "High_Price", "Low_Price", "Close_Price", "Volume"]

            # 🛠️ Fix missing dates issue: Ensure we have all dates
            df["Date"] = pd.to_datetime(df["Date"])  # Convert to datetime
            df = df.set_index("Date").asfreq("B")  # Fill missing business days
            df = df.reset_index()

            new_data.append(df)

        except Exception as e:
            print(f"⚠️ Error processing {ticker}: {e}")

except Exception as e:
    print(f"⚠️ Error fetching stock data: {e}")

# 📊 Convert the list into a DataFrame
if new_data:
    Update = pl.from_pandas(pd.concat(new_data, ignore_index=True))

    # Ensure the Date column is correctly formatted
    Update = Update.with_columns(pl.col("Date").cast(pl.Date))

    # 🔗 Ensure Update has all columns from TradeVision
    missing_cols = set(TradeVision.columns) - set(Update.columns)
    for col in missing_cols:
        Update = Update.with_columns(pl.lit(None).alias(col))

    # Reorder columns to match TradeVision
    Update = Update.select(TradeVision.columns)

    # 🔗 Merge new data with TradeVision, keeping the correct order
    TradeVision_updated = pl.concat([TradeVision, Update]).sort(["Date", "Ticker"])

    # 💾 Save the updated dataset
    output_path = r"D:\Project\TradeVision\Merged_Final_Dataset.csv"
    TradeVision_updated.write_csv(output_path)

    print(f"✅ Data updated and merged successfully! File saved at: {output_path}")
else:
    print("⚠️ No new data was downloaded!")


Fetching data for 500 tickers...
YF.download() has changed argument auto_adjust default to True
✅ Data updated and merged successfully! File saved at: D:\Project\TradeVision\Merged_Final_Dataset.csv


### 📌 STEP 1.1: Data Cleaning

####  Step 1: Load and Prepare Data

In [5]:
import pandas as pd
from tabulate import tabulate

# Load dataset and remove unnamed columns
file_path = r"D:\Project\TradeVision\Close_data.csv"

Final_Dataset = pd.read_csv(file_path, encoding='latin1', index_col=False, low_memory=False)

# Drop any columns that have "Unnamed" in their name
TradeVision = TradeVision.loc[:, ~TradeVision.columns.str.contains('^Unnamed')]

# Get dataset shape
num_rows, num_cols = TradeVision.shape

# Print dataset summary
print("\n" + "="*50)
print("✅ Dataset Successfully Loaded & Cleaned!")
print("="*50)
print(f"📊 Total Rows: {num_rows:,}")
print(f"📊 Total Columns: {num_cols}")
print("="*50)

# Print the first 10 rows in a properly formatted table
print("\n🔹 **Preview of Cleaned Dataset:**\n")
print(tabulate(TradeVision.head(10), headers="keys", tablefmt="fancy_grid", showindex=False))







✅ Dataset Successfully Loaded & Cleaned!
📊 Total Rows: 36,092
📊 Total Columns: 18

🔹 **Preview of Cleaned Dataset:**

╒════════════╤══════════╤═════════╤════════╤═══════╤════════╤══════════╤═══════════════════╤═══════════════════════════╤═════════════╤═════════════╤══════════════╤═════════╤════════════════════╤════════╤═══════════════════╤════════════════════╤═══════════════╕
│ Date       │ Ticker   │   Close │   High │   Low │   Open │   Volume │   Daily_Variation │   Cumulative_Variation_10 │   MA_10days │   MA_50days │   MA_200days │ Month   │   Pct_Change_Month │   Year │   Pct_Change_Year │   Pct_Change_Daily │ Trend_Label   │
╞════════════╪══════════╪═════════╪════════╪═══════╪════════╪══════════╪═══════════════════╪═══════════════════════════╪═════════════╪═════════════╪══════════════╪═════════╪════════════════════╪════════╪═══════════════════╪════════════════════╪═══════════════╡
│ 2020-01-04 │ AZUL4.SA │   25.13 │  25.5  │ 24.71 │  25.28 │    25.28 │         nan       │      

####  Step 2: Data Cleaning

In [ ]:
# Load dataset
Final_Dataset = pd.read_csv(r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv")

# Convert Date to datetime and remove invalid rows
Final_Dataset["Date"] = pd.to_datetime(Final_Dataset["Date"], errors="coerce", dayfirst=True)
Final_Dataset.dropna(subset=["Date"], inplace=True)

# Numeric columns that shouldn't have NaNs
num_cols = ["Open_Price", "High_Price", "Low_Price", "Close_Price", "Volume"]

# Step 1: Replace invalid values (≤ 0) with NaN
Final_Dataset[num_cols] = Final_Dataset[num_cols].where(Final_Dataset[num_cols] > 0, np.nan)

# Step 2: Fill missing values with column mean
Final_Dataset[num_cols] = Final_Dataset[num_cols].fillna(Final_Dataset[num_cols].mean())

# Convert Volume to integer (after filling NaNs)
Final_Dataset["Volume"] = Final_Dataset["Volume"].astype(int)

# Remove duplicates based on Ticker & Date
Final_Dataset.drop_duplicates(subset=["Ticker", "Date"], inplace=True)

# Final Check for Missing Data
print("\n🔍 **Missing Data in Key Columns:**")
missing_data = Final_Dataset[["Ticker", "Company", "Open_Price", "Volume"]].isnull().sum()
print(tabulate(missing_data.reset_index(), headers=["Column", "Missing Values"], tablefmt="fancy_grid"))

# Final Check for Duplicates
duplicates = Final_Dataset.duplicated().sum()
print(f"\n🔍 **Duplicated Rows After Cleaning:** {duplicates:,}")

# ✅ Display Cleaned Dataset Preview
print("\n📊 **Cleaned Dataset Preview:**\n")
print(tabulate(Final_Dataset.head(10), headers="keys", tablefmt="fancy_grid", showindex=False))

# Save the cleaned dataset
cleaned_file_path = r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset_Cleaned.csv"
Final_Dataset.to_csv(cleaned_file_path, index=False)
print(f"\n✅ **Cleaned Dataset Saved Successfully!** 📂\nPath: {cleaned_file_path}")



🔍 **Missing Data in Key Columns:**
╒════╤════════════╤══════════════════╕
│    │ Column     │   Missing Values │
╞════╪════════════╪══════════════════╡
│  0 │ Ticker     │                0 │
├────┼────────────┼──────────────────┤
│  1 │ Company    │                0 │
├────┼────────────┼──────────────────┤
│  2 │ Open_Price │                0 │
├────┼────────────┼──────────────────┤
│  3 │ Volume     │                0 │
╘════╧════════════╧══════════════════╛

🔍 **Duplicated Rows After Cleaning:** 0

📊 **Cleaned Dataset Preview:**

╒═════════════════════╤══════════╤════════════════════════════╤═════════════╤═══════════════╤══════════════╤══════════════╤═════════════╤═══════════════╤══════════╤═══════════════════╤═══════════════════════════╤═════════════╤═════════════╤══════════════╤═════════╤════════════════════╤════════╤═══════════════════╤════════════════════╤═══════════════╕
│ Date                │ Ticker   │ Company                    │ Sector      │ Country       │   Open_Price │

### 📌 STEP 1.2: Add Necessary Columns

In [19]:
import pandas as pd
from tabulate import tabulate

# Load the dataset with encoding fix
file_path = r"D:\Project\TradeVision\Close_data.csv"
Final_Dataset = pd.read_csv(file_path, encoding="utf-8")

# Print column names to check
print("\n✅ **Column Names in Dataset:**")
print(Final_Dataset.columns)

# Ensure 'Date' is in datetime format
Final_Dataset['Date'] = pd.to_datetime(Final_Dataset['Date'], errors='coerce')

# Check for NaN values in important columns
if Final_Dataset['Close'].isna().sum() > 0:
    print("⚠️ Warning: 'Close' column has missing values! Filling NaNs...")
    Final_Dataset['Close'] = Final_Dataset['Close'].fillna(method='ffill')

if Final_Dataset['Ticker'].isna().sum() > 0:
    print("⚠️ Warning: 'Ticker' column has missing values! Dropping rows...")
    Final_Dataset = Final_Dataset.dropna(subset=['Ticker'])

# Sort data
Final_Dataset = Final_Dataset.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

# 1️⃣ **Daily Variation (%)**
Final_Dataset['Daily_Variation'] = Final_Dataset.groupby('Ticker')['Close'].pct_change() * 100

# 2️⃣ **Cumulative Variation (10-day rolling sum)**
Final_Dataset['Cumulative_Variation_10'] = Final_Dataset.groupby('Ticker')['Daily_Variation'].rolling(window=10, min_periods=1).sum().reset_index(level=0, drop=True)

# 3️⃣ **Moving Averages**
Final_Dataset['MA_10days'] = Final_Dataset.groupby('Ticker')['Close'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)
Final_Dataset['MA_50days'] = Final_Dataset.groupby('Ticker')['Close'].rolling(window=50, min_periods=1).mean().reset_index(level=0, drop=True)
Final_Dataset['MA_200days'] = Final_Dataset.groupby('Ticker')['Close'].rolling(window=200, min_periods=1).mean().reset_index(level=0, drop=True)

# 4️⃣ **Monthly & Yearly Percentage Change**
Final_Dataset['Month'] = Final_Dataset['Date'].dt.to_period('M')
Final_Dataset['Pct_Change_Month'] = Final_Dataset.groupby(['Ticker', 'Month'])['Close'].pct_change() * 100

Final_Dataset['Year'] = Final_Dataset['Date'].dt.to_period('Y')
Final_Dataset['Pct_Change_Year'] = Final_Dataset.groupby(['Ticker', 'Year'])['Close'].pct_change() * 100

# 5️⃣ **Daily Percentage Change**
Final_Dataset['Pct_Change_Daily'] = Final_Dataset['Daily_Variation']

# 6️⃣ **Trend Label**
threshold = 0.5
Final_Dataset['Trend_Label'] = 'Stable'
Final_Dataset.loc[Final_Dataset['MA_10days'] > Final_Dataset['MA_50days'] + threshold, 'Trend_Label'] = 'High'
Final_Dataset.loc[Final_Dataset['MA_10days'] < Final_Dataset['MA_50days'] - threshold, 'Trend_Label'] = 'Low'

# Remove 'Unnamed' columns
Final_Dataset = Final_Dataset.loc[:, ~Final_Dataset.columns.str.contains('^Unnamed')]

# Save cleaned dataset
output_file_path = r"D:\Project\TradeVision\Close_data.csv"
Final_Dataset.to_csv(output_file_path, index=False)

# Display summary
num_rows, num_cols = Final_Dataset.shape
print("\n" + "="*60)
print(f"✅ Dataset Updated & Saved! 📊\n🔹 Total Rows: {num_rows:,} | 🔹 Total Columns: {num_cols}")
print("="*60)

# Preview dataset
print("\n📊 **Updated Dataset Preview:**\n")
print(tabulate(Final_Dataset.head(10), headers="keys", tablefmt="fancy_grid", showindex=False))



✅ **Column Names in Dataset:**
Index(['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

✅ Dataset Updated & Saved! 📊
🔹 Total Rows: 36,092 | 🔹 Total Columns: 18

📊 **Updated Dataset Preview:**

╒═════════════════════╤══════════╤═════════╤════════╤═══════╤════════╤══════════╤═══════════════════╤═══════════════════════════╤═════════════╤═════════════╤══════════════╤═════════╤════════════════════╤════════╤═══════════════════╤════════════════════╤═══════════════╕
│ Date                │ Ticker   │   Close │   High │   Low │   Open │   Volume │   Daily_Variation │   Cumulative_Variation_10 │   MA_10days │   MA_50days │   MA_200days │ Month   │   Pct_Change_Month │ Year   │   Pct_Change_Year │   Pct_Change_Daily │ Trend_Label   │
╞═════════════════════╪══════════╪═════════╪════════╪═══════╪════════╪══════════╪═══════════════════╪═══════════════════════════╪═════════════╪═════════════╪══════════════╪═════════╪════════════════════╪════════╪═══════════════════╪══════

###  Feature Engineering

In [20]:
import pandas as pd
import numpy as np

# Load dataset
Final_Dataset = pd.read_csv(r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv")

# Ensure the dataset is sorted by Date and Ticker
Final_Dataset['Date'] = pd.to_datetime(Final_Dataset['Date'], errors='coerce')
Final_Dataset.sort_values(by=['Date', 'Ticker'], inplace=True)

### 📌 1️⃣ Lag Features
Final_Dataset['Close_Price_t-1'] = Final_Dataset.groupby('Ticker')['Close_Price'].shift(1)
Final_Dataset['Close_Price_t-5'] = Final_Dataset.groupby('Ticker')['Close_Price'].shift(5)
Final_Dataset['Close_Price_t-10'] = Final_Dataset.groupby('Ticker')['Close_Price'].shift(10)

### 📌 2️⃣ Relative Strength Index (RSI - 14 days)
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain, index=series.index).rolling(window=period, min_periods=1).mean()
    avg_loss = pd.Series(loss, index=series.index).rolling(window=period, min_periods=1).mean()

    rs = avg_gain / (avg_loss + 1e-10)  # Avoid division by zero
    return 100 - (100 / (1 + rs))

Final_Dataset['RSI_14'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_rsi(x))

### 📌 3️⃣ Moving Average Convergence Divergence (MACD)
def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, adjust=False).mean()
    long_ema = series.ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    return macd_line, signal_line

# Apply MACD using transform instead of apply
Final_Dataset['MACD_Line'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_macd(x)[0])
Final_Dataset['MACD_Signal'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_macd(x)[1])
Final_Dataset['MACD_Histogram'] = Final_Dataset['MACD_Line'] - Final_Dataset['MACD_Signal']

# Debug: Check for missing values in MACD columns
print("Debug: Missing Values in MACD columns:")
print(Final_Dataset[['MACD_Line', 'MACD_Signal', 'MACD_Histogram']].isnull().sum())

### 📌 4️⃣ Bollinger Bands (20-day period)
def compute_bollinger_bands(series, window=20, num_std=2):
    sma = series.rolling(window=window, min_periods=1).mean()
    std = series.rolling(window=window, min_periods=1).std()
    upper_band = sma + (num_std * std)
    lower_band = sma - (num_std * std)
    return sma, upper_band, lower_band

# Apply Bollinger Bands using transform
Final_Dataset['Bollinger_MA'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_bollinger_bands(x)[0])
Final_Dataset['Bollinger_Upper'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_bollinger_bands(x)[1])
Final_Dataset['Bollinger_Lower'] = Final_Dataset.groupby('Ticker')['Close_Price'].transform(lambda x: compute_bollinger_bands(x)[2])

# Debug: Check for missing values in Bollinger Bands columns
print("Debug: Missing Values in Bollinger Bands columns:")
print(Final_Dataset[['Bollinger_MA', 'Bollinger_Upper', 'Bollinger_Lower']].isnull().sum())

### 📌 Save the new dataset with additional features
Final_Dataset.to_csv(r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset_Features.csv", index=False)

print("✅ Feature engineering completed! New dataset saved.")


Debug: Missing Values in MACD columns:
MACD_Line         0
MACD_Signal       0
MACD_Histogram    0
dtype: int64
Debug: Missing Values in Bollinger Bands columns:
Bollinger_MA         0
Bollinger_Upper    500
Bollinger_Lower    500
dtype: int64
✅ Feature engineering completed! New dataset saved.


### 📌 STEP 1.4: Connect DataFrame to an SQL table

In [12]:
from sqlalchemy import create_engine
import pandas as pd

# Define your database name and table
database_name = "StockData"  # Replace with actual database name
table_name = "StockData"  # Replace with actual table name

# Corrected connection string
conn_str = f"mssql+pyodbc://@localhost\\SQLExpress08/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create the SQLAlchemy engine
try:
    engine = create_engine(conn_str)
    print("✅ Engine Created!")
except Exception as e:
    print(f"❌ Error creating engine: {e}")

# Assuming your DataFrame is named 'TradeVision'
try:
    TradeVision.to_sql(table_name, con=engine, index=False, if_exists='replace')  # Use 'replace' or 'append'
    print("✅ Data uploaded successfully!")
except Exception as e:
    print(f"❌ Error uploading data: {e}")






✅ Engine Created!
✅ Data uploaded successfully!


##### //////////////////////////////////////// If Need it //////////////////////////////////////////////

### 📌 STEP 1.2: Trend Label Encoding

In [23]:
import numpy as np

# Extract 'Trend_Label' column
y = Final_Dataset['Trend_Label']

# Split the data into training and testing sets (you can adjust the test size as per your needs)
from sklearn.model_selection import train_test_split
X = Final_Dataset.drop(columns=['Trend_Label'])  # Features

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now we have y_train and y_test
print("y_train sample:", y_train.head())


y_train sample: 117597    High
164545    High
53334     High
53741      Low
30326     High
Name: Trend_Label, dtype: object


### 📌 STEP 1.3: Mapping Trend_Label to numerical values

In [24]:

trend_mapping = {'High': 1, 'Low': -1, 'Stable': 0}

# Apply the mapping to y_train and y_test
y_train_encoded = np.array([trend_mapping[label] for label in y_train])
y_test_encoded = np.array([trend_mapping[label] for label in y_test])

# Check the first few encoded values
print("Encoded y_train values:", y_train_encoded[:10])


Encoded y_train values: [ 1  1  1 -1  1  1  1  1 -1  1]


### 📌 STEP 1.4: Find Missing Values

In [25]:
import pandas as pd

# Load the dataset (adjust the path as needed)
Final_Dataset = pd.read_csv(r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv")

# Check for missing values in the dataset
missing_data = Final_Dataset.isnull().sum()

# Print missing data statistics
print("Missing Data in each column:")
print(missing_data)


Missing Data in each column:
Date                       4000
Ticker                        0
Company                       0
Open_Price                    0
High_Price                    0
Low_Price                     0
Close_Price                   0
Volume                        0
Daily_Variation               1
Cumulative_Variation_10      10
MA_10days                     9
MA_50days                    49
MA_200days                  199
Month                      4000
Pct_Change_Month           4065
Year                       4000
Pct_Change_Year            4006
Pct_Change_Daily              1
Trend_Label                   0
dtype: int64


In [24]:
# Save the Final_Dataset to a CSV file
output_file_path = r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv"
Final_Dataset.to_csv(output_file_path, index=False)

print(f"✅ Final dataset saved successfully to: {output_file_path}")


✅ Final dataset saved successfully to: D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv


#### STEP 1.2: Transform the Data to a long Format Dataset

In [ ]:
# Load the dataset (skip the first two rows, and set proper header)
df = pd.read_csv(r"D:\Project\TradeVision\Volume.csv", header=2)  # The correct header is row 3 (index 2)

# Set the first column as the 'Date' column
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

# Check the first few rows to ensure tickers and dates are properly loaded
print(df.head())

# Extract tickers from the first row (row with index 0, since header=1)
tickers = df.columns[1:]  # Get tickers from the columns, starting from the second column
dates = df['Date']  # The 'Date' column is still intact

# Prepare a list to collect the long format data
long_format_data = []

# Loop through each ticker and assign it to the dates correctly
for ticker in tickers:
    for date in dates:
        if date != 'Date':  # Skip any rows that contain the word "Date"
            # Append the date, ticker, and close value to the list
            Open_value = df.loc[df['Date'] == date, ticker].values[0]  # Get the close price for the given ticker and date
            long_format_data.append([date, ticker, Open_value])


# Create the DataFrame from the long format data
long_format_df = pd.DataFrame(long_format_data, columns=['Date', 'Ticker', 'Close'])

# Save the result to a CSV file (optional)
long_format_df.to_csv('Close_data.csv', index=False)

# Display the first few rows to check the results
print(long_format_df.head())


Poll for New Data

In [21]:
import os
import time
import pandas as pd
from SQLConnection import SQLConnection  # Import the function with the correct name

# Path to the dataset (Update.csv)
file_path = r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv"  # Use raw string for the path

# Name of the table where data will be inserted
table_name = "CreateTable"  # Replace with your actual table name in the database

def check_and_import():
    if os.path.exists(file_path):
        # Load the new data from Update.csv
        update_data = pd.read_csv(file_path)

        # Create a database connection using the function from SQLConnection.py
        conn = SQLConnection()  # Use the correct function name

        if conn:
            # Import data into the SQL table (using pyodbc)
            try:
                update_data.to_sql(table_name, conn, if_exists='append', index=False)
                print(f"Data imported successfully into {table_name}.")
            except Exception as e:
                print(f"Error while importing data: {e}")
            finally:
                # Close the connection
                conn.close()
        else:
            print("Connection to SQL Server failed.")

        # Optionally, remove the file after processing (if desired)
        os.remove(file_path)

# Polling loop: check every X seconds (e.g., 10 seconds)
while True:
    check_and_import()
    time.sleep(10)  # Wait for 10 seconds before checking again





KeyboardInterrupt: 

In [35]:
# Clean missing or invalid values (for example, replacing NaNs with None)
update_data = update_data.fillna({'Column_Name': None})  # Replace with None for columns that can accept NULL

# Ensure numeric columns are correct type
update_data["Open_Price"] = pd.to_numeric(update_data["Open_Price"], errors='coerce')
update_data["Close_Price"] = pd.to_numeric(update_data["Close_Price"], errors='coerce')
update_data["Volume"] = pd.to_numeric(update_data["Volume"], errors='coerce')
# You can apply this similarly to other numerical columns

# Re-check the DataFrame after cleaning
print(update_data.head())


         Date Ticker Company  Open_Price  High_Price   Low_Price  Close_Price  \
0  2020-02-01    MMM      3M  120.780382  122.364235  120.413308   122.357437   
1  2020-03-01    MMM      3M  120.331736  121.446543  119.386858   121.303795   
2  2020-06-01    MMM      3M  120.420099  121.480533  119.876285   121.419350   
3  2020-07-01    MMM      3M  121.188228  121.344585  120.195775   120.929924   
4  2020-08-01    MMM      3M  120.997903  123.377075  120.759980   122.785675   

    Volume  Daily_Variation  Cumulative_Variation_10  MA_10days  MA_50days  \
0  4307633              NaN                      NaN        NaN        NaN   
1  2950412        -0.861118                      NaN        NaN        NaN   
2  2389608         0.095261                      NaN        NaN        NaN   
3  2598908        -0.403087                      NaN        NaN        NaN   
4  3298927         1.534567                      NaN        NaN        NaN   

   MA_200days    Year    Month  Pct_Change_D

In [38]:
# Fill NaN values in the 'Year' column with a placeholder (e.g., 0)
update_data["Year"] = update_data["Year"].fillna(0).astype(int)

# Convert 'Month' to an integer (extract the month)
update_data["Month"] = pd.to_datetime(update_data["Month"], errors='coerce').dt.month

# Fill NaN values for numerical columns with zeros (or None for nullable fields)
update_data[["Daily_Variation", "Cumulative_Variation_10", "MA_10days", "MA_50days", "MA_200days"]] = \
    update_data[["Daily_Variation", "Cumulative_Variation_10", "MA_10days", "MA_50days", "MA_200days"]].fillna(0)

# For Trend_Label, if NaN values exist, fill with 'Unknown'
update_data["Trend_Label"].fillna("Unknown", inplace=True)

# Now continue with the rest of the code...




C:\Users\cleid\AppData\Local\Temp\ipykernel_17052\3903671677.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  update_data["Trend_Label"].fillna("Unknown", inplace=True)


In [44]:
# Fill NaN values in 'Month' with 0 or any other valid integer
update_data["Month"] = update_data["Month"].fillna(0).astype(int)




In [41]:
# Check Datatype

# Check data types of all columns in the DataFrame
print(update_data.dtypes)


Date                        object
Ticker                      object
Company                     object
Open_Price                 float64
High_Price                 float64
Low_Price                  float64
Close_Price                float64
Volume                       int64
Daily_Variation            float64
Cumulative_Variation_10    float64
MA_10days                  float64
MA_50days                  float64
MA_200days                 float64
Year                         int64
Month                      float64
Pct_Change_Daily           float64
Pct_Change_Month           float64
Pct_Change_Year            float64
Trend_Label                 object
dtype: object


In [46]:
import pyodbc
import pandas as pd
import numpy as np

# Establish connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost\\SQLExpress08;'  # Adjust server and instance if needed
    'DATABASE=StockData;'              # Ensure this is the correct database
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# Load new data from your Update.csv
update_data = pd.read_csv(r"D:\Project\TradeVision\Data\Final Dataset\Final_Dataset.csv")

# Ensure column names match SQL table
expected_columns = [
    "Date", "Ticker", "Company", "Open_Price", "High_Price", "Low_Price", "Close_Price",
    "Volume", "Daily_Variation", "Cumulative_Variation_10", "MA_10days", "MA_50days",
    "MA_200days", "Year", "Month", "Pct_Change_Daily", "Pct_Change_Month",
    "Pct_Change_Year", "Trend_Label"
]

# Select only the expected columns
update_data = update_data[expected_columns]

# Fill NaN values in the 'Year' column with a placeholder (e.g., 0)
update_data["Year"] = update_data["Year"].fillna(0).astype(int)

# Convert 'Month' to an integer (extract the month)
update_data["Month"] = pd.to_datetime(update_data["Month"], errors='coerce').dt.month

# Fill NaN values for numerical columns with zeros (or None for nullable fields)
update_data[["Daily_Variation", "Cumulative_Variation_10", "MA_10days", "MA_50days", "MA_200days"]] = \
    update_data[["Daily_Variation", "Cumulative_Variation_10", "MA_10days", "MA_50days", "MA_200days"]].fillna(0)

# For Trend_Label, if NaN values exist, fill with 'Unknown'
update_data["Trend_Label"].fillna("Unknown", inplace=True)

# Convert Date column to correct format (if needed)
update_data["Date"] = pd.to_datetime(update_data["Date"]).dt.date

# Ensure float columns do not have invalid values, convert invalid entries to None
float_columns = ["Open_Price", "High_Price", "Low_Price", "Close_Price", "Daily_Variation", 
                 "Cumulative_Variation_10", "MA_10days", "MA_50days", "MA_200days", 
                 "Pct_Change_Daily", "Pct_Change_Month", "Pct_Change_Year"]

# Replace any invalid values (e.g., NaN, infinity) with None (null values)
update_data[float_columns] = update_data[float_columns].apply(pd.to_numeric, errors='coerce')

# Convert DataFrame to list of tuples for bulk insert
data_tuples = [tuple(x) for x in update_data.to_numpy()]

# SQL Query with correct table name (StockData)
sql = """
    INSERT INTO StockData (
        Date, Ticker, Company, Open_Price, High_Price, Low_Price, Close_Price,
        Volume, Daily_Variation, Cumulative_Variation_10, MA_10days, MA_50days,
        MA_200days, Year, Month, Pct_Change_Daily, Pct_Change_Month, 
        Pct_Change_Year, Trend_Label
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Enable fast execution for bulk insert
cursor.fast_executemany = True

# Execute bulk insert
cursor.executemany(sql, data_tuples)

# Commit and close
conn.commit()
conn.close()

print("✅ Bulk insert completed successfully!")


C:\Users\cleid\AppData\Local\Temp\ipykernel_17052\353439332.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  update_data["Trend_Label"].fillna("Unknown", inplace=True)


ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 16 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecute)')